In [ ]:
%%javascript

require(['widgets/js/widget', 'widgets/js/manager', 'jquery', 'underscore'], function(widget, manager, $, _) {
    /**
     * A Google Maps API widget.
     *
     * @constructor
     */
    var GoogleMapsView = widget.DOMWidgetView.extend({
        /**
         * Load the Maps API JS if needed, also prepare a deferred in case any
         * map methods are called before the map is ready.
         */
        initialize: function() {
            // Deferred to track for when the map is ready.
            this.mapsReadyDeferred = $.Deferred();

            // Dynamically adding Google Maps API JS here. Using a deferred to
            // track its load status as require returns as soon as the first
            // script loads and the Maps API triggers more scripts to append
            // which leaves a race condition where require thinks Maps API JS
            // is ready when it is not yet.
            var mapsDeferred = this.mapsDeferred = $.Deferred();
            // Another instance of this view may have already loaded the Maps
            // API JS, do not try to load it twice.
            if (window.google && window.google.maps) {
                mapsDeferred.resolve();
            } else {
                window.googleMapsCallback = function() {
                    mapsDeferred.resolve();
                };
                require(
                    ['http://maps.googleapis.com/maps/api/js?callback=googleMapsCallback'],
                    function() {},
                    function() {});
            }
        },

        /**
         * Render the map.
         */
        render: function() {
            // We must wait until the Maps API JS is ready.
            $.when(this.mapsDeferred.promise()).then(_.bind(function() {
                // Empty the views DOM. There seem to be some weird side
                // effects when you render more than one instance of this view
                // in the notebook. Cleaning the view DOM and deferring the map
                // initialization seems to work around this. It seesm almost if
                // map instances are sharing some DOM somehow.
                this.$el.empty();
                _.defer(_.bind(function() {
                    this.$map = $([
                        '<div style="height: ',
                        this.model.get('height'),
                        '; width: ',
                        this.model.get('width'),
                        ';"></div>'
                    ].join(''));
                    this.$el.append(this.$map);
                    this.map = new google.maps.Map(this.$map.get(0), {
                        center: {
                            lat: this.model.get('lat'),
                            lng: this.model.get('lng')
                        },
                        zoom: this.model.get('zoom')
                    });

                    // Notify the map when the map container changes size via
                    // the exposed properties in the model.
                    this.listenTo(
                        this.model, 'change:height', _.bind(function() {
                            this.$map.height(this.model.get('height'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));
                    this.listenTo(
                        this.model, 'change:width', _.bind(function() {
                            this.$map.width(this.model.get('width'));
                            google.maps.event.trigger(this.map, 'resize');
                        }, this));

                    // Bind a change in the position of the map to the model.
                    google.maps.event.addListener(
                        this.map,
                        'bounds_changed',
                        _.bind(this.syncFromMap, this));

                    // Bind a change in the model (coming from the Python side)
                    // to the location of the map.
                    this.listenTo(
                        this.model,
                        'change:lat',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:lng',
                        _.bind(this.syncFromModel, this));
                    this.listenTo(
                        this.model,
                        'change:zoom',
                        _.bind(this.syncFromModel, this));

                    // Broker messages coming from the Python side over to
                    // methods on the view.
                    this.listenTo(
                        this.model,
                        'msg:custom',
                        _.bind(this.handleMessage, this));

                    // Even though a google.maps.Map instance should be ready
                    // immediately, it is not. This delay lets the stack clear
                    // and initial map bounds to be set.
                    _.delay(_.bind(function() {
                        this.mapsReadyDeferred.resolve();
                    }, this), 500);
                }, this));
            }, this));
        },

        /**
         * Sync the values from the map into the model.
         */
        syncFromMap: function() {
            this.model.set({
                lat: this.map.getCenter().lat(),
                lng: this.map.getCenter().lng(),
                zoom: this.map.getZoom()
            });
            // This is needed for the model to update the equivalent properties
            // on the Python instance of this view.
            this.model.save_changes();
        },

        /**
         * Move the map to match the values from the model.
         */
        syncFromModel: function() {
            this.map.setCenter(new google.maps.LatLng(
                this.model.get('lat'),
                this.model.get('lng')
            ));
            this.map.setZoom(this.model.get('zoom'));
        },

        /**
         * Process a message from the Python view instance and call the desired
         * Javascript method.
         *
         * This method expects a dictionary sent by Python using self.send. The
         * dictionary should look like:
         *
         *   {'method': 'jsMethodToCall', 'params': [1, 2, 3, 4]}
         *
         * The list of parameters is optional and will be passed in the same
         * order to the Javascript method named by the 'method' property.
         *
         * This is an example of a simple passthrough:
         *
         *   def fooBar(self, param1, param2):
         *       self.send({'method': 'fooBar', 'params': [param1, param2]})
         *
         * The above would excute the following in Javascript:
         *
         *   this.fooBar(param1, param2);
         *
         * @param {!Object<string, *>} payload The message payload describing
         *     the method to call.
         */
        handleMessage: function(payload) {
            var params = payload.params || [];
            this[payload.method].apply(this, params);
        },

        /**
         * Add an Earth Engine layer to the map.
         *
         * @param {string} mapid The id of the Earth Engine layer.
         * @param {string} token The OAuth token to authenticate with.
         * @param {string} name The name to use for the layer.
         * @param {boolean} visible Whether the layer should be visible or not.
         */
        addLayer: function(mapid, token, name, visible) {
            // Defer until map is ready.
            this.mapsReadyDeferred.done(_.bind(function() {
                var eeMapOptions = {
                    getTileUrl: function(tile, zoom) {
                        var url = [
                            'https://earthengine.googleapis.com/map',
                            mapid,
                            zoom,
                            tile.x,
                            tile.y
                        ].join('/');
                        url += '?token=' + token;
                        return url;
                    },
                    tileSize: new window.google.maps.Size(256, 256),
                    opacity: visible ? 1.0 : 0.0,
                };

                // Create the overlay map type.
                var mapType = new window.google.maps.ImageMapType(eeMapOptions);

                // Overlay the Earth Engine generated layer.
                this.map.overlayMapTypes.push(mapType);
            }, this));
        }
    });

    // Register the view.
    manager.WidgetManager.register_widget_view(
        'GoogleMapsView', GoogleMapsView);
});

In [ ]:
from ipywidgets import widgets
import traitlets

class GoogleMapsView(widgets.DOMWidget):
    """Google Maps API widget."""
    _view_name = traitlets.Unicode('GoogleMapsView', sync=True)
    lat = traitlets.CFloat(0, sync=True)
    lng = traitlets.CFloat(0, sync=True)
    zoom = traitlets.CInt(2, sync=True)
    
    height = traitlets.CUnicode('300px', sync=True)
    width = traitlets.CUnicode('400px', sync=True)
        
    def addLayer(self, image, vis_params):
        """Adds an Earth Engine layer to the map.
        
        Args:
            image: The ee.Image to display.
            vis_params: Dictionary of visualization parameters.
        """
        mapid = image.getMapId(vis_params)
        self.send({
            'method': 'addLayer',
            'params': [
                mapid['mapid'], 
                mapid['token'],
                'dummy',
                True
            ]
        })

In [ ]:
from IPython.display import display
import ee

l8 = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2013-05-01', '2013-05-15').mosaic()
dem = ee.Image('USGS/SRTMGL1_003')

vis_params_dem = {'min': 0, 'max': 3000}
vis_params_true = {'min': 0, 'max': 0.3, 'bands': 'B4,B3,B2'}
vis_params_false = {'min': 0, 'max': 0.3, 'bands': 'B5,B4,B3'}

# Overview map.
map1 = GoogleMapsView(lng=-119.2, lat=36.3, zoom=4, height='200px', width='600px')

# SRTM
map2 = GoogleMapsView(zoom=6, height='200px', width='600px')

# Landsat 8 True Color
map3 = GoogleMapsView(height='200px', width='600px')

# Landsat 8 False Color
map4 = GoogleMapsView(height='200px', width='600px')

In [ ]:
# Link up the interactive content.
widgets.jslink((map1, 'lat'), (map2, 'lat'))
widgets.jslink((map1, 'lat'), (map3, 'lat'))
widgets.jslink((map1, 'lat'), (map4, 'lat'))
widgets.jslink((map1, 'lng'), (map2, 'lng'))
widgets.jslink((map1, 'lng'), (map3, 'lng'))
widgets.jslink((map1, 'lng'), (map4, 'lng'))
widgets.jslink((map2, 'zoom'), (map3, 'zoom'))
widgets.jslink((map2, 'zoom'), (map4, 'zoom'))

In [ ]:
display(map1, map2, map3, map4)
# Overlay some Earth Engine data.
map2.addLayer(image=dem, vis_params=vis_params_dem)
map3.addLayer(image=l8, vis_params=vis_params_true)
map4.addLayer(image=l8, vis_params=vis_params_false)